In [10]:
# Mengimpor library yang dibutuhkan
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Memuat model BERT pre-trained
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Memuat data
data = [
    "2 onion$diabetic$Blood cholesterol was lowered significantly by dietary ##### in ######## animals.",
    "0 Garlic$cancer$ ###### consumption and ###### prevention: meta-analyses of colorectal and stomach ######s.",
    "1 garlic$tumor$The ##### incidences were 3% and 9% in animals given AFB1 plus ###### and AFB1 plus ###### oil, respectively.",
    "0 coffee$gallbladder disease$The authors examined the relation of ultrasound-documented ################### with ###### drinking in 13,938 adult participants in the Third National Health and Nutrition Examination Survey, 1988-1994."
]
df = pd.DataFrame(data, columns=["text"])

# Menambahkan kolom untuk label entitas dan atribut
df["entity_label"] = np.nan
df["attribute_label"] = np.nan

# Melakukan tokenisasi pada setiap kalimat dalam dataset
def tokenize_text(text):
    tokens = tokenizer.encode_plus(text, max_length=128, truncation=True,
                                   padding="max_length", return_tensors="tf")
    return {
        "input_ids": tf.cast(tokens["input_ids"], tf.int32),
        "attention_mask": tf.cast(tokens["attention_mask"], tf.int32),
        "token_type_ids": tf.cast(tokens["token_type_ids"], tf.int32),
    }

# Menerapkan model pada setiap kalimat dalam dataset
def predict_class(text):
    tokens = tokenize_text(text)
    output = model(tokens)[0]
    return tf.argmax(output, axis=1).numpy()[0]

# Menyimpan label entitas dan atribut pada setiap kalimat dalam dataset
for i in range(len(df)):
    text = df.loc[i, "text"]
    label = predict_class(text)
    entity, attribute, _ = text.split("$")
    df.loc[i, "entity_label"] = label
    df.loc[i, "attribute_label"] = label

# Menampilkan hasil ekstraksi multi-hubungan
print(df)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

ImportError: 
TFAutoModelForSequenceClassification requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.
